Shallow Learning

In [1]:
import numpy as np
import pickle
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import numpy as np
import pandas as pd
import torch
from gensim.models import Word2Vec
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from gensim.models import Word2Vec
from tensorflow.keras.utils import to_categorical
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score

In [5]:
def shallow_pipeline(df, target_col):
    from collections import Counter
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder
    from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
    from sklearn.linear_model import LogisticRegression
    from sklearn.svm import LinearSVC
    from sklearn.metrics import accuracy_score, f1_score
    
    # Preparar texto
    if "text_joined" not in df.columns:
        df["text_joined"] = df["tokens"].apply(lambda x: " ".join(x))
    texts = df["text_joined"].astype(str).tolist()
    labels = df[target_col].tolist()
    
    # Codificar etiquetas
    le = LabelEncoder()
    y = le.fit_transform(labels)
    
    # Revisar si se puede hacer stratify
    min_count = min(Counter(y).values())
    stratify_arg = y if min_count > 1 else None
    
    # Split
    X_train_text, X_val_text, y_train, y_val = train_test_split(
        texts, y, test_size=0.2, random_state=42, stratify=stratify_arg
    )
    
    results = {}
    
    def train_eval(model, X_train, X_val, name=""):
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        acc = accuracy_score(y_val, preds)
        f1 = f1_score(y_val, preds, average="macro")
        print(f"{name} → Accuracy: {acc:.4f} | Macro-F1: {f1:.4f}")
        return acc, f1
    
    # BoW
    vectorizer_bow = CountVectorizer(max_features=20000)
    X_train_bow = vectorizer_bow.fit_transform(X_train_text)
    X_val_bow = vectorizer_bow.transform(X_val_text)
    
    results["BoW_LogReg"] = train_eval(
        LogisticRegression(max_iter=3000), X_train_bow, X_val_bow, "BoW + LogisticRegression"
    )
    
    results["BoW_SVM"] = train_eval(
        LinearSVC(), X_train_bow, X_val_bow, "BoW + LinearSVM"
    )
    
    # TF-IDF Unigram
    vectorizer_tfidf_uni = TfidfVectorizer(max_features=20000, ngram_range=(1,1))
    X_train_tfidf_uni = vectorizer_tfidf_uni.fit_transform(X_train_text)
    X_val_tfidf_uni = vectorizer_tfidf_uni.transform(X_val_text)
    
    results["TFIDF_uni_LogReg"] = train_eval(
        LogisticRegression(max_iter=3000), X_train_tfidf_uni, X_val_tfidf_uni, "TF-IDF Unigram + LogisticRegression"
    )
    
    results["TFIDF_uni_SVM"] = train_eval(
        LinearSVC(), X_train_tfidf_uni, X_val_tfidf_uni, "TF-IDF Unigram + LinearSVM"
    )
    
    # TF-IDF N-grams (1–2)
    vectorizer_tfidf_ngram = TfidfVectorizer(max_features=40000, ngram_range=(1,2))
    X_train_tfidf_ngram = vectorizer_tfidf_ngram.fit_transform(X_train_text)
    X_val_tfidf_ngram = vectorizer_tfidf_ngram.transform(X_val_text)
    
    results["TFIDF_ngram_LogReg"] = train_eval(
        LogisticRegression(max_iter=3000), X_train_tfidf_ngram, X_val_tfidf_ngram, "TF-IDF N-gram + LogisticRegression"
    )
    
    results["TFIDF_ngram_SVM"] = train_eval(
        LinearSVC(), X_train_tfidf_ngram, X_val_tfidf_ngram, "TF-IDF N-gram + LinearSVM"
    )
    
    return results


In [ ]:
results_topic = shallow_pipeline(df, "topic")


In [ ]:
results_bias = shallow_pipeline(df, "bias")



=== Ejecutando Shallow Learning para topic ===


KeyboardInterrupt: 

In [ ]:
results_source = shallow_pipeline(df, "source")

Deep Learning

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, f1_score
from gensim.models import Word2Vec

# -----------------------------
# Función para entrenar y evaluar
# -----------------------------
def deep_learning_task(task_label, df, w2v_model, max_seq_len=100, epochs=3, batch_size=64):
    print(f"\n=== Tarea: {task_label} ===")
    
    # Datos y etiquetas
    texts = df["tokens"].tolist()
    labels = df[task_label].tolist()
    
    le = LabelEncoder()
    y = le.fit_transform(labels)
    
    # Split train/val
    X_tr_text, X_val_text, y_tr, y_val = train_test_split(
        texts, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Word2Vec embeddings
    embedding_dim = w2v_model.vector_size
    word_index = {word: i+1 for i, word in enumerate(w2v_model.wv.index_to_key)}
    vocab_size = len(word_index) + 1
    
    def tokens_to_indices(tokens, word_index):
        return [word_index[t] for t in tokens if t in word_index]
    
    X_tr_idx = [tokens_to_indices(t, word_index) for t in X_tr_text]
    X_val_idx = [tokens_to_indices(t, word_index) for t in X_val_text]
    
    X_tr_pad = pad_sequences(X_tr_idx, maxlen=max_seq_len, padding='post')
    X_val_pad = pad_sequences(X_val_idx, maxlen=max_seq_len, padding='post')
    
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, i in word_index.items():
        embedding_matrix[i] = w2v_model.wv[word]
    
    # Función para crear modelo
    def build_rnn(model_type='LSTM', embedding_matrix=None, trainable=True):
        model = Sequential()
        model.add(Embedding(input_dim=embedding_matrix.shape[0],
                            output_dim=embedding_matrix.shape[1],
                            weights=[embedding_matrix],
                            input_length=max_seq_len,
                            trainable=trainable))
        if model_type == 'LSTM':
            model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
        elif model_type == 'GRU':
            model.add(GRU(64, dropout=0.2, recurrent_dropout=0.2))
        model.add(Dense(len(np.unique(y)), activation='softmax'))
        model.compile(optimizer=Adam(1e-3),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        return model
    
    # Entrenamiento y evaluación
    results = {}
    for model_type in ['LSTM', 'GRU']:
        for emb_type, trainable, emb_matrix in [
            ('Word2Vec Frozen', False, embedding_matrix),
            ('Word2Vec Fine-tune', True, embedding_matrix),
            ('Word2Vec Scratch', True, np.random.normal(size=(vocab_size, embedding_dim)))
        ]:
            print(f"\nEntrenando {model_type} con {emb_type}...")
            model = build_rnn(model_type=model_type, embedding_matrix=emb_matrix, trainable=trainable)
            model.fit(X_tr_pad, y_tr, validation_data=(X_val_pad, y_val),
                      epochs=epochs, batch_size=batch_size, verbose=1)
            preds = np.argmax(model.predict(X_val_pad, batch_size=batch_size), axis=1)
            acc = accuracy_score(y_val, preds)
            f1 = f1_score(y_val, preds, average='macro')
            results[f"{model_type} + {emb_type}"] = (acc, f1)
            print(f"{model_type} + {emb_type} → Accuracy: {acc:.4f} | Macro-F1: {f1:.4f}")
    
    # Mostrar resultados
    results_df = pd.DataFrame(results, index=['Accuracy','Macro-F1']).T
    print("\n==============================")
    print(f"RESULTADOS TAREA: {task_label}")
    print("==============================")
    print(results_df)
    return results_df

# -----------------------------
# 2. Cargar dataset y Word2Vec
# -----------------------------
df = pd.read_pickle("data/data_clean/train_tokenized.pkl")
w2v_model = Word2Vec.load("data/embeddings/word2vec.model")




In [ ]:
results_bias = deep_learning_task("bias", df, w2v_model)

In [ ]:
results_medio = deep_learning_task("source", df, w2v_model)

In [ ]:
results_tema = deep_learning_task("topic", df, w2v_model)